# Part 9 Machine Learning With Spark ML
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

---
# 1. Get the data
We start by loading the dataset. we infer column types automatically by reading the filewith `inferSchema` to true. The `header` option will read the columns' name from the file.

In [57]:
val default = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("data/ccdefault.csv")

default: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]


---
# 2. Discover the data to gain insights
here we analyse the data by obtaining the statistical summary of the attributes. We attempt to find the correlation between the different features. In the end, we combine 18 out of the 24 features in 3 groups of 6 to generate 3 new attribues.

## 2.1. Schema and dimension
Print the schema of the dataset

In [58]:
default.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

Print the number of records in the dataset.

In [59]:
default.count()

res43: Long = 30000


## 2.2. Look at the data
Print the first five records of the dataset.

In [60]:
default.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|     3455|     3261|       0|    1000|    1000|    1000|       0|    2000|    

Print the number of records with age more than 30.

In [61]:
default.filter("age > 30").count()

res45: Long = 18987


## 2.3. Statistical summary
A summary of the table statistics for the attributes `age`, `sex`, `education`, and `limit_bal`.

In [62]:
// TODO: Replace <FILL IN> with appropriate code

default.describe("age", "sex", "education", "limit_bal").show()

+-------+-----------------+------------------+------------------+------------------+
|summary|              age|               sex|         education|         limit_bal|
+-------+-----------------+------------------+------------------+------------------+
|  count|            30000|             30000|             30000|             30000|
|   mean|          35.4855|1.6037333333333332|1.8531333333333333|167484.32266666667|
| stddev|9.217904068090155|0.4891291960902602|0.7903486597207269|129747.66156720246|
|    min|               21|                 1|                 0|             10000|
|    max|               79|                 2|                 6|           1000000|
+-------+-----------------+------------------+------------------+------------------+



In [63]:
default.groupBy("DEFAULT").count().show()

default.groupBy("SEX").count().show()

+-------+-----+
|DEFAULT|count|
+-------+-----+
|      1| 6636|
|      0|23364|
+-------+-----+

+---+-----+
|SEX|count|
+---+-----+
|  1|11888|
|  2|18112|
+---+-----+



## 2.4. Correlation among attributes
Correlation between the attributes `age`, `sex`, `education`, and `limit_bal`,by computing the standard correlation coefficient (Pearson) between every pair.

In [64]:
import org.apache.spark.ml.feature.VectorAssembler

val va = new VectorAssembler().setInputCols(Array("LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE")).setOutputCol("features")

val defaultAttrs = va.transform(default)

defaultAttrs.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+--------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|DEFAULT|            features|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-------+--------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|      1|[20000.0,2.0,2.0,...|
|  2|   120000|  2|        2|       2| 26|   -1|    2|    0|    0|    0|    2|     2682|     1725|     2682|    

import org.apache.spark.ml.feature.VectorAssembler
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_66b0513e16a5
defaultAttrs: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 24 more fields]


In [65]:
import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row

val Row(coeff: Matrix) = Correlation.corr(defaultAttrs, "features").head

println(s"The standard correlation coefficient:\n ${coeff}")

The standard correlation coefficient:
 1.0                   0.024755235111645853  -0.2191606982292233   ... (5 total)
0.024755235111645853  1.0                   0.01423193616219367   ...
-0.2191606982292233   0.01423193616219367   1.0                   ...
-0.10813941027800818  -0.03138884007083411  -0.14346434041145634  ...
0.14471279755736938   -0.09087364652720994  0.17506066148814436   ...


import org.apache.spark.ml.linalg.Matrix
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.sql.Row
coeff: org.apache.spark.ml.linalg.Matrix =
1.0                   0.024755235111645853  -0.2191606982292233   ... (5 total)
0.024755235111645853  1.0                   0.01423193616219367   ...
-0.2191606982292233   0.01423193616219367   1.0                   ...
-0.10813941027800818  -0.03138884007083411  -0.14346434041145634  ...
0.14471279755736938   -0.09087364652720994  0.17506066148814436   ...


## 2.5. Combine and make new attributes
We chose to combine the mopnthly payment parameters and to average them. We generate 3 new features, `repayment`, `billStatement`, and `payment`

In [66]:
import org.apache.spark.sql.functions._

val repayCol = Array(col("PAY_0"), col("PAY_2"), col("PAY_3"), col("PAY_4"), col("PAY_5"), col("PAY_6"))
val averageRepay = repayCol.foldLeft(lit(0)){(x, y) => x+y}/repayCol.length
val default1 = default.withColumn("repayment", averageRepay)


val billCol = Array(col("BILL_AMT1"), col("BILL_AMT2"), col("BILL_AMT3"), col("BILL_AMT4"), col("BILL_AMT5"), col("BILL_AMT6"))
val averageBill = billCol.foldLeft(lit(0)){(x, y) => x+y}/billCol.length
val default2 = default1.withColumn("billStatement", averageBill)

val payCol = Array(col("PAY_AMT1"), col("PAY_AMT2"), col("PAY_AMT3"), col("PAY_AMT4"), col("PAY_AMT5"), col("PAY_AMT6"))
val averagePay = payCol.foldLeft(lit(0)){(x, y) => x+y}/payCol.length
val defaultExtra = default2.withColumn("payment", averagePay)

defaultExtra.select("repayment", "billStatement", "payment").show(5)


+-------------------+------------------+------------------+
|          repayment|     billStatement|           payment|
+-------------------+------------------+------------------+
|-0.3333333333333333|            1284.0|114.83333333333333|
|                0.5|2846.1666666666665| 833.3333333333334|
|                0.0|16942.166666666668|1836.3333333333333|
|                0.0|38555.666666666664|            1398.0|
|-0.3333333333333333|18223.166666666668|            9841.5|
+-------------------+------------------+------------------+
only showing top 5 rows



import org.apache.spark.sql.functions._
repayCol: Array[org.apache.spark.sql.Column] = Array(PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6)
averageRepay: org.apache.spark.sql.Column = (((((((0 + PAY_0) + PAY_2) + PAY_3) + PAY_4) + PAY_5) + PAY_6) / 6)
default1: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 24 more fields]
billCol: Array[org.apache.spark.sql.Column] = Array(BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6)
averageBill: org.apache.spark.sql.Column = (((((((0 + BILL_AMT1) + BILL_AMT2) + BILL_AMT3) + BILL_AMT4) + BILL_AMT5) + BILL_AMT6) / 6)
default2: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 25 more fields]
payCol: Array[org.apache.spark.sql.Column] = Array(PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)
averagePay...

---
## 3. Preparing the data for Machine Learning algorithms


In [67]:
val renamedDefault = default.withColumnRenamed("DEFAULT", "label")

val filteredDefault =renamedDefault.drop("ID")

renamedDefault: org.apache.spark.sql.DataFrame = [ID: int, LIMIT_BAL: int ... 23 more fields]
filteredDefault: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 22 more fields]


In [68]:
// label columns
val colLabel = "label"

// categorical columns
val colCat1 = "SEX"
val colCat2 = "EDUCATION"
val colCat3 = "MARRIAGE"

// numerical columns
val colNum = filteredDefault.columns.filter(_ != colLabel).filter(_ != colCat1).filter(_ != colCat2).filter(_ != colCat3)

colLabel: String = label
colCat1: String = SEX
colCat2: String = EDUCATION
colCat3: String = MARRIAGE
colNum: Array[String] = Array(LIMIT_BAL, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6)


## 3.1. Prepare continuse attributes
### Data cleaning
The dataset does not have missing values.

In [69]:
// TODO: Replace <FILL IN> with appropriate code

for (c <- colNum) {
    println(c, filteredDefault.filter(filteredDefault(c).isNull || filteredDefault(c) === "" || filteredDefault(c).isNaN).count())
}

(LIMIT_BAL,0)
(AGE,0)
(PAY_0,0)
(PAY_2,0)
(PAY_3,0)
(PAY_4,0)
(PAY_5,0)
(PAY_6,0)
(BILL_AMT1,0)
(BILL_AMT2,0)
(BILL_AMT3,0)
(BILL_AMT4,0)
(BILL_AMT5,0)
(BILL_AMT6,0)
(PAY_AMT1,0)
(PAY_AMT2,0)
(PAY_AMT3,0)
(PAY_AMT4,0)
(PAY_AMT5,0)
(PAY_AMT6,0)


### Scaling
Here we standardize the values of the attributes so that the resulting distribution has unit variance.

In [70]:
import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}

val va = new VectorAssembler().setInputCols(colNum).setOutputCol("featuresscale")
val featuredDefault = va.transform(filteredDefault) 
val scaler = new StandardScaler().setInputCol("featuresscale").setOutputCol("scaled")
val scaledDefault = scaler.fit(featuredDefault).transform(featuredDefault)

scaledDefault.show(5)
scaledDefault.columns

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|       featuresscale|              scaled|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
|   120000|  2|        2|       2| 26|   -1|    2|  

import org.apache.spark.ml.feature.{VectorAssembler, StandardScaler}
va: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_5302515b6f43
featuredDefault: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 23 more fields]
scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_666aae1a126a
scaledDefault: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 24 more fields]
res52: Array[String] = Array(LIMIT_BAL, SEX, EDUCATION, MARRIAGE, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, label, featuresscale, scaled)


## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. 

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [71]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

val encoder = new OneHotEncoderEstimator().setInputCols(Array("SEX", "EDUCATION", "MARRIAGE")).setOutputCols(Array("oh-sex", "oh-education", "oh-marriage"))
val ohDefault = encoder.fit(filteredDefault).transform(filteredDefault)

ohDefault.show(5)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------+-------------+-------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|       oh-sex| oh-education|  oh-marriage|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+-------------+-------------+-------------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|    (2,[],[])|(6,[2],[1.0])|(3,[1],[1.0])|
|   120000|  2|        2|       2| 26|   -1|    2|  

import org.apache.spark.ml.feature.OneHotEncoderEstimator
encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_5ca6fc8d8f46
ohDefault: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 25 more fields]


---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [72]:
import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}

val numPipeline = new Pipeline().setStages(Array(va, scaler))
val catPipeline = new Pipeline().setStages(Array(encoder))
val pipeline = new Pipeline().setStages(Array(numPipeline, catPipeline))
val newDefault = pipeline.fit(filteredDefault).transform(filteredDefault)
newDefault.show(5)

+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+-------------+-------------+-------------+
|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|       featuresscale|              scaled|       oh-sex| oh-education|  oh-marriage|
+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+-------------+-------------+-------------+
|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|  

import org.apache.spark.ml.{Pipeline, PipelineModel, PipelineStage}
numPipeline: org.apache.spark.ml.Pipeline = pipeline_be27b93ea66f
catPipeline: org.apache.spark.ml.Pipeline = pipeline_e2fbb4215819
pipeline: org.apache.spark.ml.Pipeline = pipeline_db9cac09a62b
newDefault: org.apache.spark.sql.DataFrame = [LIMIT_BAL: int, SEX: int ... 27 more fields]


Creating the `dataset` to be used. 

In [73]:
val va2 = new VectorAssembler().setInputCols(Array("scaled", "oh-sex", "oh-education", "oh-marriage")).setOutputCol("features")
val dataset = va2.transform(newDefault).select("features", "label")

dataset.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(31,[0,1,2,3,4,5,...|    1|
|(31,[0,1,2,3,7,8,...|    1|
|(31,[0,1,8,9,10,1...|    0|
|(31,[0,1,8,9,10,1...|    0|
|(31,[0,1,2,4,8,9,...|    0|
+--------------------+-----+
only showing top 5 rows



va2: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_e4687b21f7c2
dataset: org.apache.spark.sql.DataFrame = [features: vector, label: int]


---
# 5. Make a model
Here we going to implement four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression


In [74]:
val Array(trainSet, testSet) = dataset.randomSplit(Array(0.8, 0.2))

trainSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]
testSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [features: vector, label: int]


## 5.1. Logistic regression model
Now, train a Linear Regression model using the `LogisticRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `binarySummary` method.

In [75]:
import org.apache.spark.ml.classification.LogisticRegression

// train the model
val lr = new LogisticRegression().setMaxIter(10)
val lrModel = lr.fit(trainSet)
val trainingSummary = lrModel.binarySummary

val roc = trainingSummary.roc
roc.show()

println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")
println(s"AreaUnderROC: ${trainingSummary.areaUnderROC}")

+--------------------+-------------------+
|                 FPR|                TPR|
+--------------------+-------------------+
|                 0.0|                0.0|
|0.003048128342245989|0.03404017857142857|
|0.006524064171122995|0.06677827380952381|
|0.009786096256684492|0.10007440476190477|
|0.013101604278074866| 0.1331845238095238|
|0.016844919786096257|0.16480654761904762|
|0.020106951871657754|0.19810267857142858|
| 0.02374331550802139| 0.2300967261904762|
|0.028449197860962568| 0.2583705357142857|
| 0.03374331550802139| 0.2845982142857143|
| 0.03935828877005348|0.30970982142857145|
|0.044598930481283425| 0.3361235119047619|
|0.050160427807486634|0.36142113095238093|
| 0.05711229946524064|0.38188244047619047|
| 0.06379679144385027|0.40327380952380953|
| 0.07181818181818182|0.42001488095238093|
| 0.08010695187165776|0.43582589285714285|
| 0.08871657754010695| 0.4505208333333333|
| 0.09780748663101604| 0.4640997023809524|
| 0.10647058823529412|0.47898065476190477|
+----------

import org.apache.spark.ml.classification.LogisticRegression
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_72d648d33267
lrModel: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_72d648d33267, numClasses = 2, numFeatures = 31
trainingSummary: org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummary = org.apache.spark.ml.classification.BinaryLogisticRegressionTrainingSummaryImpl@26e86458
roc: org.apache.spark.sql.DataFrame = [FPR: double, TPR: double]


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [76]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator


// make predictions on the test data
val predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

val evaluator = new BinaryClassificationEvaluator().setMetricName("areaUnderROC").setRawPredictionCol("prediction").setLabelCol("label")
val roc = evaluator.evaluate(predictions)
println(s"AreaUnderROC on test data = $roc")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(31,[0,1,2,3,4,5,...|
|       1.0|    1|(31,[0,1,2,3,4,5,...|
|       0.0|    1|(31,[0,1,2,3,4,5,...|
|       1.0|    1|(31,[0,1,2,3,4,5,...|
|       0.0|    0|(31,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows

AreaUnderROC on test data = 0.6087577255030086


import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 3 more fields]
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_4d11b19fce0d
roc: Double = 0.6087577255030086


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [77]:
import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val dt = new DecisionTreeRegressor().setLabelCol("label").setFeaturesCol("features")

// train the model
val dtModel = dt.fit(trainSet)

// make predictions on the test data
val predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(10)

// select (prediction, true label) and compute test error
val evaluator = new BinaryClassificationEvaluator().setMetricName("areaUnderROC").setRawPredictionCol("prediction").setLabelCol("label")
val roc = evaluator.evaluate(predictions)
println(s"AreaUnderROC on test data = $roc")

+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
|         0.19921875|    0|(31,[0,1,2,3,4,5,...|
| 0.8571428571428571|    1|(31,[0,1,2,3,4,5,...|
|0.14072289156626505|    1|(31,[0,1,2,3,4,5,...|
| 0.7975528364849833|    1|(31,[0,1,2,3,4,5,...|
|0.12633451957295375|    0|(31,[0,1,2,3,4,5,...|
|0.31016731016731014|    0|(31,[0,1,2,3,4,5,...|
| 0.7975528364849833|    1|(31,[0,1,2,3,4,5,...|
|0.31016731016731014|    1|(31,[0,1,2,3,4,5,...|
|0.12633451957295375|    0|(31,[0,1,2,3,4,5,...|
|0.12633451957295375|    0|(31,[0,1,2,3,4,5,...|
+-------------------+-----+--------------------+
only showing top 10 rows

AreaUnderROC on test data = 0.7490702169947453


import org.apache.spark.ml.regression.DecisionTreeRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
dt: org.apache.spark.ml.regression.DecisionTreeRegressor = dtr_4eec98790d7c
dtModel: org.apache.spark.ml.regression.DecisionTreeRegressionModel = DecisionTreeRegressionModel (uid=dtr_4eec98790d7c) of depth 5 with 61 nodes
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_e59e3098c1f8
roc: Double = 0.7490702169947453


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [78]:
// TODO: Replace <FILL IN> with appropriate code

import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator

val rf = new RandomForestRegressor().setLabelCol("label").setFeaturesCol("features")

// train the model
val rfModel = rf.fit(trainSet)

// make predictions on the test data
val predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

// select (prediction, true label) and compute test error
val evaluator = new BinaryClassificationEvaluator().setMetricName("areaUnderROC").setRawPredictionCol("prediction").setLabelCol("label")
val roc = evaluator.evaluate(predictions)
println(s"AreaUnderROC on test data = $roc")

+-------------------+-----+--------------------+
|         prediction|label|            features|
+-------------------+-----+--------------------+
|0.20657230524875209|    0|(31,[0,1,2,3,4,5,...|
| 0.7123833221279636|    1|(31,[0,1,2,3,4,5,...|
|0.15715065434373532|    1|(31,[0,1,2,3,4,5,...|
| 0.7667315257069983|    1|(31,[0,1,2,3,4,5,...|
| 0.2939590654911295|    0|(31,[0,1,2,3,4,5,...|
+-------------------+-----+--------------------+
only showing top 5 rows

AreaUnderROC on test data = 0.7652918164121001


import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
rf: org.apache.spark.ml.regression.RandomForestRegressor = rfr_954d11776507
rfModel: org.apache.spark.ml.regression.RandomForestRegressionModel = RandomForestRegressionModel (uid=rfr_954d11776507) with 20 trees
predictions: org.apache.spark.sql.DataFrame = [features: vector, label: int ... 1 more field]
evaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_5d51b1bb61cd
roc: Double = 0.7652918164121001


## The models we used

We tested three different models to predict default: Logistic Regression, Desicion Tree and Random Forest. The results were as expected, they were improving in this respective order. We used AUC to compare the different approaches. The distribution of the dataset was not equal so the result depends largely on the test set that is chosen. Logistic Regression's linear boundaries generalizes better and have less chance of overfitting compared to Decision Trees, but also is less accurate. Decision Trees are simple and explainable models and already perform quite well. But Random Forests will always yield better accuracy and is more robust as it's constructed of multiple decision trees. 

AreaUnderROC for test sets:

Logistic Regression: 0.6087577255030086

Decision Tree: 0.7490702169947453

Random Forest: 0.7652918164121001

The given attributes are relevant with the problem of financial default, and all the models achieved an accuracy of over 60%.
The models could become more accurate if we obtained more historical data and would train on more aggregate features. More granular data for parameters like education would also help to improve the predictions.